In [240]:
import io
from datetime import datetime, timedelta

import psycopg
import xarray as xr

In [10]:
pg_connection_dict = {
    'dbname': "mydb",
    'user': "myn",
    'password': r"IE>AUs]6)fr>jQ8m",
    'port': "5432",
    'host': "terraform-20221218184631507400000002.c2x7llrlmsr3.us-east-2.rds.amazonaws.com"
}

with psycopg.connect(**pg_connection_dict, autocommit=True) as conn:
    with conn.cursor() as curr:
        print(conn.info.encoding)
        print(curr.execute(r"SELECT now()").fetchall())

utf-8
[(datetime.datetime(2022, 12, 18, 19, 29, 16, 761866, tzinfo=datetime.timezone.utc),)]


In [246]:
with psycopg.connect(**pg_connection_dict, autocommit=True) as conn:
    with conn.cursor() as curr:
        curr.execute(open("../database.sql", "rt").read())

In [247]:
ds = xr.open_dataset(
    "../data/CMC_reg_SNOD_SFC_0_ps10km_2022121518_P001.grib2",
    engine="cfgrib",
    indexpath="",
)

In [114]:
ds["time"]

<xarray.DataArray 'time' ()>
[1 values with dtype=datetime64[ns]]
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
    valid_time  datetime64[ns] ...
Attributes:
    long_name:      initial time of forecast
    standard_name:  forecast_reference_time

In [61]:
ds_var = list(ds.keys())[0]

variable = {
        "short_name": ds[ds_var].attrs["GRIB_shortName"],
        "long_name": ds[ds_var].attrs["GRIB_name"],
        "unit": ds[ds_var].attrs["units"],
    }


variable

{'short_name': 'sde', 'long_name': 'Snow depth', 'unit': 'm'}

In [60]:
variable

{'short_name': str, 'long_name': str, 'unit': str}

In [248]:
with psycopg.connect(**pg_connection_dict, autocommit=True) as conn:
    with conn.cursor() as curr:
        curr.execute("""
        INSERT INTO public.variables(short_name, long_name, unit) VALUES (
            %(short_name)s,
            %(long_name)s,
            %(unit)s
        );""", variable)

In [50]:
ds.attrs

{'GRIB_edition': 2,
 'GRIB_centre': 'cwao',
 'GRIB_centreDescription': 'Canadian Meteorological Service - Montreal',
 'GRIB_subCentre': 0,
 'Conventions': 'CF-1.7',
 'institution': 'Canadian Meteorological Service - Montreal',
 'history': '2022-12-18T14:54 GRIB to CDM+CF via cfgrib-0.9.10.3/ecCodes-2.27.0 with {"source": "../data/CMC_reg_SNOD_SFC_0_ps10km_2022121518_P000.grib2", "filter_by_keys": {}, "encode_cf": ["parameter", "time", "geography", "vertical"]}'}

In [51]:
ds["step"].values

numpy.timedelta64(0,'ns')

In [249]:
from datetime import datetime, timezone
import numpy as np
from numpy import typing as npt


def to_utcdatetime(date: np.datetime64):
    """
    Converts a numpy datetime64[ns] object to a python datetime object
    Input:
      date - a np.datetime64[ns] object
    Output:
      DATE - a python datetime object
    """
    timestamp = int((date - np.datetime64("1970-01-01T00:00:00")) / np.timedelta64(1, "s"))
    return datetime.fromtimestamp(timestamp, timezone.utc)


print(to_utcdatetime(ds["time"].values))

def to_timedelta(td: np.timedelta64):
    """
    Converts a numpy timedelta64[ns] object to a python timedelta object
    Input:
      date - a np.datetime64[ns] object
    Output:
      DATE - a python datetime object
    """
    return timedelta(td.astype("timedelta64[h]").astype("float") / 24)

print(to_timedelta(ds["step"].values))

2022-12-15 18:00:00+00:00
1:00:00


In [137]:
ds["time"].values

numpy.datetime64('2022-12-15T18:00:00.000000000')

In [112]:
timedelta(int(ds["step"].values.astype("timedelta64[ms]").astype("int")))

datetime.timedelta(0)

In [250]:
forecast = {
    "model": "rdps",
    "reference_time": to_utcdatetime(ds["time"].values),
    "step": to_timedelta(ds["step"].values),
}

with psycopg.connect(**pg_connection_dict, autocommit=True) as conn:
    with conn.cursor() as curr:
        curr.execute("""
        INSERT INTO public.forecasts(model, forecast_reference_time, forecast_step) VALUES (
            %(model)s,
            %(reference_time)s,
            %(step)s
        );""", forecast)

In [186]:
df = ds.to_dataframe().fillna({ds_var: 0})
new_df = df[["latitude", "longitude", ds_var]].reset_index(drop=True)
new_df = new_df.rename(columns={ds_var: "value"})
new_df["model"] = "rdps"
new_df["forecast_reference_time"] = to_utcdatetime(ds["time"].values)
new_df["forecast_step"] = to_timedelta(ds["step"].values)
new_df["short_name"] = ds[ds_var].attrs["GRIB_shortName"]
new_df = new_df[["model", "forecast_reference_time", "forecast_step", "short_name", "longitude", "latitude", "value"]]

In [ ]:
list(new_df.itertuples(index=False))

In [223]:
import pandas as pd
write_df = new_df.loc[:, ["longitude", "latitude", "value"]]
write_df["forecast_id"] = 1
write_df["variable_id"] = 2
write_df = write_df.loc[:, ["forecast_id", "variable_id", "longitude", "latitude", "value"]]
write_df

,forecast_id,variable_id,longitude,latitude,value
0,1,2,217.107456,18.145030,0.0
1,1,2,217.163959,18.178401,0.0
2,1,2,217.220531,18.211734,0.0
3,1,2,217.277173,18.245027,0.0
4,1,2,217.333884,18.278282,0.0
...,...,...,...,...,...
770435,1,2,349.914588,45.729948,0.0
770436,1,2,349.892200,45.648763,0.0
770437,1,2,349.869903,45.567618,0.0
770438,1,2,349.847696,45.486515,0.0


In [224]:
write_query = """
INSERT INTO public.predictions(forecast_id, variable_id, coordinates, value) VALUES (
	%s,
	%s,
	point(%s, %s),
	%s
)"""

In [228]:
with psycopg.connect(**pg_connection_dict, autocommit=True) as conn:
    with conn.cursor() as curr:
        with conn.transaction():
            curr.execute("""
                SELECT forecast_id
                FROM public.forecasts
                WHERE 
                    model = %(model)s AND
                    forecast_reference_time = %(reference_time)s AND
                    forecast_step = %(step)s
            """, forecast)
        forecast_id = curr.fetchone()[0]

        with conn.transaction():
            curr.execute("""
                SELECT variable_id
                FROM public.variables
                WHERE short_name = %s
            """, [variable["short_name"]])
        variable_id = curr.fetchone()[0]

        # Prepare data to upload
        df = ds.to_dataframe().fillna({ds_var: 0})
        df = df[["longitude", "latitude", ds_var]].reset_index(drop=True)
        df = df.loc[:, ["longitude", "latitude", ds_var]]
        df["forecast_id"] = forecast_id
        df["variable_id"] = variable_id
        df = df.loc[:, ["forecast_id", "variable_id", "longitude", "latitude", ds_var]]

        curr.executemany(write_query, list(df.itertuples(index=False)))

In [ ]:
with psycopg.connect(**pg_connection_dict, autocommit=True) as conn:
    with conn.cursor() as curr:
        with conn.transaction():
            curr.execute("""
                SELECT forecast_id
                FROM public.forecasts
                WHERE 
                    model = %(model)s AND
                    forecast_reference_time = %(reference_time)s AND
                    forecast_step = %(step)s
            """, forecast)
        forecast_id = curr.fetchone()[0]

        with conn.transaction():
            curr.execute("""
                SELECT variable_id
                FROM public.variables
                WHERE short_name = %s
            """, [variable["short_name"]])
        variable_id = curr.fetchone()[0]

        # Prepare data to upload
        df = ds.to_dataframe().fillna({ds_var: 0})
        df = df[["longitude", "latitude", ds_var]].reset_index(drop=True)
        df = df.loc[:, ["longitude", "latitude", ds_var]]
        df["forecast_id"] = forecast_id
        df["variable_id"] = variable_id
        df = df.loc[:, ["forecast_id", "variable_id", "longitude", "latitude", ds_var]]
        csv_file_like_object = io.StringIO()
        df.to_csv(csv_file_like_object, header=False, index=False)
        csv_file_like_object.seek(0)

        with curr.copy("""
            COPY public.predictions(forecast_id, variable_id, longitude, latitude, value) 
            FROM STDIN""") as copy:
            lines = csv_file_like_object.read().splitlines()
            for line in lines:
                print(line.split(","))
                copy.write_row(line)

In [237]:

df = ds.to_dataframe().fillna({ds_var: 0})
df = df[["longitude", "latitude", ds_var]].reset_index(drop=True)
df = df.loc[:, ["longitude", "latitude", ds_var]]
df["forecast_id"] = 1
df["variable_id"] = 2
df = df.loc[:, ["forecast_id", "variable_id", "longitude", "latitude", ds_var]]
csv_file_like_object = io.StringIO()
df.to_csv(csv_file_like_object, header=False, index=False)
csv_file_like_object.seek(0)

In [ ]:
csv_file_like_object.readlines()